In [2]:
import numpy as np
import pandas as pd
import json
from sklearn.decomposition import PCA
import plotly.express as px

# --- 1) Carica la matrice di co-occorrenza ---
cxc = pd.read_csv("cxc.csv", header=None).values

# --- 2) Carica dizionario carte ---
with open('./royale/cards_dict.json', "r", encoding="utf-8") as f:
    cards_dict = json.load(f)

for i, (cid, c) in enumerate(cards_dict.items()):
    c['loc'] = i   # aggiungi la posizione
    
with open('./royale/cards_dict.json', "w", encoding="utf-8") as f:
    json.dump(cards_dict, f, indent=2, ensure_ascii=False)

# creo un mapping indice -> nome carta
idx_to_label = [None] * len(cards_dict)
for cid, info in cards_dict.items():
    idx = info['loc']   # se nel tuo dict hai la posizione della carta
    idx_to_label[idx] = info.get('name', cid)

# --- 3) PCA in 3 dimensioni ---
pca = PCA(n_components=3)
coords = pca.fit_transform(cxc)

# --- 4) Etichette ---
labels = idx_to_label

# --- 5) Visualizza ---
fig = px.scatter_3d(
    x=coords[:,0],
    y=coords[:,1],
    z=coords[:,2],
    text=labels,
    color=coords[:,0],
    title="Embedding 3D delle carte (PCA su co-occorrenze)"
)
fig.update_traces(marker=dict(size=5))
fig.show()
